In [489]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, timedelta

# 設置MySQL資料庫連接
db_user = 'root'
db_password = '19970730'
db_host = '127.0.0.1'
db_name = 'sql_stock'

# 創建資料庫連接引擎
engine = create_engine(f'mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}')

# 設置日期範圍
#today = datetime(2024, 5, 25)
today = datetime.now().date()
one_year_ago = today - timedelta(days=365)

# 格式化日期
today_str = today.strftime('%Y%m%d')
one_year_ago_str = one_year_ago.strftime('%Y%m%d')

# SQL 查詢 加權指數
query_twse = f"""
SELECT *
FROM daily_twse
WHERE date >= '{one_year_ago_str}' AND date <= '{today_str}'
"""

# SQL 查詢 個股
query_stock = f"""
SELECT *
FROM daily_price
WHERE date >= '{one_year_ago_str}' AND date <= '{today_str}'
"""

# SQL 查詢 細產業
query_sub_category = f"""
SELECT *
FROM sub_category
"""

# 使用 pd.read_sql 來執行查詢並讀取數據到 DataFrame
twse_df_raw = pd.read_sql(query_twse, engine)

stock_df_raw = pd.read_sql(query_stock, engine)

sub_category_list = pd.read_sql(query_sub_category, engine)



In [462]:
import pandas as pd
import pymysql

class MySQLUpdater:
    def __init__(self, db_params, table_name, df_stocks, condition, method):
        self.db = pymysql.connect(**db_params)
        self.cursor = self.db.cursor()
        self.table_name = table_name
        self.df_stocks = df_stocks
        self.condition = condition
        self.method = method
        


    def update_records(self):
        # 遍历 DataFrame 中的每一行

        for i, row in self.df_stocks.iterrows():
            try:
                if(self.method == '1'):
                    # 构建 SQL 更新语句
                    sql = f"""
                    UPDATE `{self.table_name}`
                    SET `5MA` = %s,
                        `8MA` = %s,
                        `10MA` = %s,
                        `20MA` = %s,
                        `20RS` = %s,
                        `60MA` = %s,
                        `60RS` = %s
                    WHERE `{self.condition}` = %s AND `Date` = %s
                    """
                    # 执行更新语句
                    self.cursor.execute(sql, (float(row['5MA']),float(row['8MA']),float(row['10MA']),float(row['20MA']),float(row['20RS']),float(row['60MA']),float(row['60RS']), row[self.condition], row['Date']))

                    # 提交更改
                    self.db.commit()
                    
                elif (self.method == '2'):

                    # 构建 SQL 更新语句 更新標準化RS
                    sql = f"""
                    UPDATE `{self.table_name}`
                    SET `20RS_sd` = %s,
                        `20RS_rank` = %s
                    WHERE `{self.condition}` = %s AND `Date` = %s
                    """

                    # 执行更新语句
                    self.cursor.execute(sql, (float(row['20RS_sd']),float(row['20RS_rank']), row[self.condition], row['Date']))

                    # 提交更改
                    self.db.commit()

            except Exception as e:
                print(f"Error updating row {i}: {e}")
                self.db.rollback()

    def close_connection(self):
        self.cursor.close()
        self.db.close()

# 示例使用
db_params = {
    'host': '127.0.0.1',
    'user': 'root',
    'password': '19970730',
    'database': 'sql_stock'
    # 設置MySQL資料庫連接
}

In [465]:
twse_df = twse_df_raw[twse_df_raw['指數名稱'] == '發行量加權股價指數'].copy()

twse_df_na = twse_df.isna()
twse_df_update = twse_df[twse_df_na.any(axis=1)] #需要被更新的欄位

twse_df.loc[:, '5MA'] = twse_df['價格指數值'].rolling(5).mean()
twse_df.loc[:, '8MA'] = twse_df['價格指數值'].rolling(8).mean()
twse_df.loc[:, '10MA'] = twse_df['價格指數值'].rolling(10).mean()
twse_df.loc[:, '20MA']= twse_df['價格指數值'].rolling(20).mean()
twse_df.loc[:, '60MA']= twse_df['價格指數值'].rolling(60).mean()

twse_df.loc[:, '20RS'] = round((twse_df['價格指數值']/twse_df['20MA'])*100, 4)
twse_df.loc[:, '60RS'] = round((twse_df['價格指數值']/twse_df['60MA'])*100, 4)
twse_df = twse_df.fillna(0)
#twse_df_new = twse_df.iloc[-2:]
twse_df_new = twse_df.loc[twse_df_update.index]  # 只更新尚未計算的新欄位


# 初始化并更新数据库
updater = MySQLUpdater(db_params, 'daily_twse', twse_df_new, '指數名稱', '1')
updater.update_records()
updater.close_connection()

In [440]:
stock_ids = stock_df_raw['證券代號'].unique().tolist()

stock_df_na = stock_df_raw.isna()
stock_df_update = stock_df_raw[stock_df_na.any(axis=1)] #需要被更新的欄位

for i in stock_ids:
        if((len(i)==4) ): #1. 排除債 ETF等等 2.排除新掛牌資料不到calculate_day天
                #print(i)
                # 計算個股 RS
                temp_df = stock_df_raw[stock_df_raw['證券代號']==i].copy()

                temp_df.loc[:, '5MA'] = temp_df['收盤價'].rolling(5).mean()
                temp_df.loc[:, '8MA'] = temp_df['收盤價'].rolling(8).mean()
                temp_df.loc[:, '10MA'] = temp_df['收盤價'].rolling(10).mean()
                temp_df.loc[:, '20MA']= temp_df['收盤價'].rolling(20).mean()
                temp_df.loc[:, '60MA']= temp_df['收盤價'].rolling(60).mean()

                temp_df.loc[:, '20RS'] = round((temp_df['收盤價']/temp_df['20MA'])*100, 4)
                temp_df.loc[:, '60RS'] = round((temp_df['收盤價']/temp_df['60MA'])*100, 4)
                temp_df = temp_df.fillna(0)

                #只更新尚未update欄位
                temp_update_df = stock_df_update[stock_df_update['證券代號']==i].copy()
                temp_df_new = temp_df.loc[temp_update_df.index]  # 只更新尚未計算的新欄位
                # 初始化并更新数据库
                updater = MySQLUpdater(db_params, 'daily_price', temp_df_new, '證券代號','1')
                updater.update_records()
updater.close_connection()

In [507]:
# 計算個股對大盤RS
#先轉置取Date
stock_df_raw['Date'] = pd.to_datetime(stock_df_raw['Date'])
stock_date_list = stock_df_raw['Date'].unique().tolist()

twse_df = twse_df_raw[twse_df_raw['指數名稱'] == '發行量加權股價指數'].copy()
twse_df['Date'] = pd.to_datetime(twse_df['Date'])
twse_df.rename(columns={'20RS': '20RS_twse'}, inplace=True)
cols = ['Date', '價格指數值', '20RS_twse']
twse_df_forRS = twse_df[cols]
twse_df_forRS

merged_df = pd.merge(stock_df_raw, twse_df_forRS, on='Date', how='inner')
merged_df['20RS_sd'] = merged_df['20RS']/merged_df['20RS_twse']  #20MA計算RS
merged_df = merged_df.fillna(0)

col2 = ['Date', '證券代號', '20RS_sd','20RS_rank']
for i in range(len(stock_date_list)) :
    temp_merged_df = merged_df[merged_df['Date']==stock_date_list[i]].copy()
    # #標準化成0-100名
    temp_merged_df['20RS_rank'] = (temp_merged_df['20RS_sd'].rank(axis=0,method='first')/len(temp_merged_df))*100
    temp_merged_df = temp_merged_df[col2]

    # 初始化并更新数据库
    updater = MySQLUpdater(db_params, 'daily_price', temp_merged_df, '證券代號','2')
    updater.update_records()
updater.close_connection()



# df_rs_week = pd.concat([df_rs_week,df_rs70])
# df_rsall_week = pd.concat([df_rsall_week,df_rs])
# # 順便統計細產業佔比
# rs70_sub_category = get_sub_category_list(sub_category_list,df_rs70,day)                        
# df_rs70_category_week = pd.concat([df_rs70_category_week,rs70_sub_category])


,Date,證券代號,20RS_sd,20RS_rank
66094,2024-05-24,0050,1.012048,80.890337
66095,2024-05-24,0051,0.987617,67.752443
66096,2024-05-24,0052,1.020945,83.496200
66097,2024-05-24,0053,1.018759,83.116178
66098,2024-05-24,0055,0.976331,57.926167
...,...,...,...,...
67931,2024-05-24,9951,0.961888,39.413681
67932,2024-05-24,9955,1.035488,87.024973
67933,2024-05-24,9958,1.193790,98.154180
67934,2024-05-24,9960,0.967359,47.394137


In [302]:
# 取得細產業資料
def get_sub_category_list(df_sub_category_list, df_rs70, date):
    import collections
    sub_industry_list=[]
    for i in df_rs70.index:
        try:
            temp = df_sub_category_list.loc[i].sub_category

            if(isinstance(temp, str)):
                sub_industry_list.append(temp)
            else:
                for j in temp:
                    sub_industry_list.append(j)                    
        except:
            pass
            #print(df_rs70.loc[i]['stock_name'])

    df_rs70_sub_category = collections.Counter(sub_industry_list)
    sub_category_list = collections.Counter(df_sub_category_list['sub_category'])

    df_sub_category = pd.DataFrame()
    for i in df_rs70_sub_category:
        sub_category = {
        "date": [date],
        "sub_category": [i],
        "main_rank": [df_rs70_sub_category[i]],
        "all": [sub_category_list[i]],
        "pen": [round((df_rs70_sub_category[i]/sub_category_list[i])*100,2)]}

        df_sub_category_temp = pd.DataFrame(sub_category)
        df_sub_category = pd.concat([df_sub_category,df_sub_category_temp])
    df_sub_category.sort_values(by=['pen'], ascending=False, inplace=True)

    return df_sub_category


In [305]:
# 計算RS ROLLING 用 月/季/年
count_day = 5
select_date = '2024-05-16'

twse_df['Date'] = pd.to_datetime(twse_df['Date'])

# 開始算大盤的RS
twse_df = twse_df.set_index(['指數名稱','Date'])
twse_df = twse_df.loc['發行量加權股價指數']
twse_df = twse_df.rename(columns={
    '價格指數值':'point',
    '報酬指數值':'pn',
    '漲跌點數':'diff',
    '漲跌百分比':'diff_per'
})

twse_df = twse_df[['point','pn','diff','diff_per']]
twse_df.sort_values(by=['Date'], inplace=True)
twse_df['MA'] = twse_df['point'].rolling(count_day).mean()
twse_df['RS'] = round((twse_df['point']/twse_df['MA'])*100, 4)
#twe_df


# 假设您要提取的日期是 '2024-05-16'
stock_df['Date'] = pd.to_datetime(stock_df['Date'])
# 使用 .loc 方法根据日期提取所有相关的股票数据
stock_df_temp = stock_df[stock_df['Date'] == select_date]['證券代號']
stock_ids = stock_df_temp.tolist()


# 開始算個股RS
stock_df = stock_df.set_index(['證券代號','Date'])
stock_df = stock_df.rename(columns={
    '證券名稱':'stock_name',
    '開盤價':'open',
    '最高價':'high',
    '最低價':'low',
    '收盤價':'close',
    '成交金額':'volume'
})

stock_df = stock_df[['stock_name' ,'open','high','low','close','volume']]
# 避免報錯

stock_df= stock_df.apply(lambda s:s.astype(str).str.replace(',',''))
cols_to_convert = ['open', 'high', 'low', 'close', 'volume']
stock_df[cols_to_convert] = stock_df[cols_to_convert].apply(pd.to_numeric, errors='coerce')
stock_df = stock_df.loc[stock_df['volume']>0]



In [199]:
import collections
import datetime

#讀取細產業list

sub_category_list = sub_category_list.set_index(['stock_id'])

df_rs_week = pd.DataFrame()
df_rs70_category_week = pd.DataFrame()


df_rsall_week = pd.DataFrame()
df_rsall_category_week = pd.DataFrame()

# 計算RS ROLLING 用 月/季/年
calculate_day = 5 #往前找N個交易日
countdown = calculate_day

rs_rank = 90 # 只挑選RS大於70


# 取得最新上市櫃股票ID
#stock_ids = getstock_id_list(end)

In [205]:
import collections
import datetime


# 計算RS日期 直接用 start 日期
# 開始計算 一週RS
data_date_list = pd.to_datetime(stock_df.index.get_level_values('Date')).sort_values(ascending=False).unique().tolist()

for day in data_date_list:
        day = day.strftime('%Y-%m-%d')
        
        countdown -= 1 
        df_rs = pd.DataFrame()
        if(countdown>0):           
                for i in stock_ids:
                        if((len(i)==4) & (len(stock_df.loc[i])> calculate_day)): #1. 排除債 ETF等等 2.排除新掛牌資料不到calculate_day天
                                #print(i)
                                # 計算個股 RS
                                temp_df = stock_df.loc[i]
                                #temp_df.sort_values(by=['date'], inplace=True)
                                temp_df['MA'] = temp_df['close'].rolling(count_day).mean()
                                temp_df['RS'] = round((temp_df['close']/temp_df['MA'])*100, 4)
                                # RS計算
                                stock_ma = temp_df.loc[day,'MA']
                                stock_rs = temp_df.loc[day,'RS']
                                stock_close = temp_df.loc[day,'close']
                                stock_name = temp_df.loc[day,'stock_name']
                                rs = {
                                "stock_id": [i],
                                "stcok_name" : [stock_name],
                                "date": [day],
                                "ma": [stock_ma],
                                "rs": [stock_rs],
                                "close": [stock_close]}
                                
                                df_rs_temp = pd.DataFrame(rs)
                                #print(df_rs_temp)
                                df_rs = pd.concat([df_rs,df_rs_temp])

                twe_rs = {
                "stock_id": ['twe'],
                "date": [day],
                "ma": [twse_df.loc[day,'MA']],
                "rs": [twse_df.loc[day,'RS']],
                "close": [twse_df.loc[day,'point']]}

                twe_rs_temp = pd.DataFrame(twe_rs)
                df_rs = pd.concat([df_rs,twe_rs_temp])
                df_rs = df_rs.set_index(['stock_id'])
                print(day)

                # 拿出大盤資料
                twe_rs = df_rs.loc['twe','rs']
                df_rs['rs'] = df_rs['rs']/twe_rs # 要直接除以大盤 做標準
                df_rs = df_rs.sort_values(by=['rs'], ascending=False)
                df_rs = df_rs.dropna(axis=0,how='any')
                #標準化成0-100名
                df_rs['rank'] = (df_rs['rs'].rank(axis=0,method='first')/len(df_rs))*100

                df_rs70 = df_rs[df_rs['rank']>=rs_rank]
                df_rs70['stock_name'] = ''

                
                df_rs_week = pd.concat([df_rs_week,df_rs70])
                df_rsall_week = pd.concat([df_rsall_week,df_rs])
                # 順便統計細產業佔比
                rs70_sub_category = get_sub_category_list(sub_category_list,df_rs70,day)                        
                df_rs70_category_week = pd.concat([df_rs70_category_week,rs70_sub_category])

        else:
                break


compare_df = pd.DataFrame()
countdown = calculate_day
data_date = pd.to_datetime(stock_df.index.get_level_values('Date')).sort_values(ascending=False)
data_date_list = collections.Counter(data_date)
for day in list(data_date_list):
        day_change = day.strftime('%Y-%m-%d')
        countdown -= 1 
        df_rs = pd.DataFrame()
        if(countdown>0):   
                getIndex = list(data_date_list).index(day) # 取index 找下一天
                day_before = list(data_date_list)[getIndex+1].strftime('%Y-%m-%d')
                df_rs70_category_1 = df_rs70_category_week[(df_rs70_category_week['date'] == day_change) &(df_rs70_category_week['main_rank'] > 2) & (df_rs70_category_week['pen'] > 20)]
                df_rs70_category_2 = df_rs70_category_week[(df_rs70_category_week['date'] == day_before) &(df_rs70_category_week['main_rank'] > 2) & (df_rs70_category_week['pen'] > 20)]

                sub_category_list = list(df_rs70_category_1['sub_category'])
                sub_category_before = list(df_rs70_category_2['sub_category'])
                compare_add = [x for x in sub_category_list if x not in sub_category_before] #新進入名單
                compare_remove = [x for x in sub_category_before if x not in sub_category_list] #新進入名單
                compare_list = {
                "date": [day_change],
                "add": [compare_add],
                "remove": [compare_remove]}

                compare_list_temp = pd.DataFrame(compare_list)
                compare_df = pd.concat([compare_df,compare_list_temp])

C:\Users\USER\AppData\Local\Temp\ipykernel_11044\3787636852.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['MA'] = temp_df['close'].rolling(count_day).mean()
C:\Users\USER\AppData\Local\Temp\ipykernel_11044\3787636852.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['RS'] = round((temp_df['close']/temp_df['MA'])*100, 4)


KeyboardInterrupt: 

ValueError: True is not in list

In [260]:
def date_changer(date):

    year = date[:4]
    year = str(int(year)-1911)
    month = date[4:6]
    day = date[6:]

    return year+"/"+month+"/"+day

def Rename_df_columns(df, Flag_tpex_stocks = False, Flag_tpex_insti_inv = False):

        tpex_stocks_rename_columns = {  "代號":"證券代號",
                                        "名稱":"證券名稱",
                                        "收盤 ":"收盤價",
                                        "漲跌":"漲跌價差",
                                        "開盤 ":"開盤價", 
                                        "最高 ":"最高價",
                                        "最低":"最低價",
                                        "成交股數  ":"成交股數",
                                        "成交金額(元)":"成交金額",
                                        "成交筆數 ":"成交筆數"}

        tpex_insti_inv_rename_columns = {   "代號":"證券代號", 
                                            "名稱":"證券名稱", 
                                            "外資及陸資(不含外資自營商)-買進股數":"外陸資買進股數(不含外資自營商)", 
                                            "外資及陸資(不含外資自營商)-賣出股數":"外陸資賣出股數(不含外資自營商)", 
                                            "外資及陸資(不含外資自營商)-買賣超股數":"外陸資買賣超股數(不含外資自營商)", 
                                            "外資自營商-買進股數":"外資自營商買進股數", 
                                            "外資自營商-賣出股數":"外資自營商賣出股數", 
                                            "外資自營商-買賣超股數":"外資自營商買賣超股數",
                                            "投信-買進股數":"投信買進股數",
                                            "投信-賣出股數":"投信賣出股數",
                                            "投信-買賣超股數":"投信買賣超股數",
                                            "自營商(自行買賣)-買進股數":"自營商買進股數(自行買賣)",
                                            "自營商(自行買賣)-賣出股數":"自營商賣出股數(自行買賣)",
                                            "自營商(自行買賣)-買賣超股數":"自營商買賣超股數(自行買賣)",
                                            "自營商(避險)-買進股數":"自營商買進股數(避險)",
                                            "自營商(避險)-賣出股數":"自營商賣出股數(避險)",
                                            "自營商(避險)-買賣超股數":"自營商買賣超股數(避險)",
                                            "自營商-買賣超股數":"自營商買賣超股數",
                                            "三大法人買賣超股數合計":"三大法人買賣超股數" }

        if Flag_tpex_stocks:  
            df.rename(columns=tpex_stocks_rename_columns, inplace = True)
        elif Flag_tpex_insti_inv:
            df.rename(columns=tpex_insti_inv_rename_columns, inplace = True)
        else:
            print("Error!!")

        return df

In [276]:
import requests
from io import StringIO
url_tpex_stock = "http://www.tpex.org.tw/web/stock/aftertrading/daily_close_quotes/stk_quote_download.php?l=zh-tw&d="
        # self.tpex_df_stocks = pd.DataFrame( data = [], 

#url_suffix = '&type=ALLBUT0999'
url_suffix='&s=0,asc,0'
r = requests.get( url_tpex_stock + '113/05/14' + url_suffix)
df = pd.read_csv(StringIO(r.text), header=2).dropna(how='all', axis=1).dropna(how='any')

df = df.iloc[:, :11]

df = Rename_df_columns(df, Flag_tpex_stocks = True, Flag_tpex_insti_inv = False)

#df = self.Get_specific_stock(df)

df.insert(0, "Date", '20240514')

df = df[df['證券代號'].apply(lambda x: len(x) == 4)]

df.drop("均價 ", axis = "columns", inplace = True)

df["漲跌(+/-)"] = df["漲跌價差"].values[0][0] if df["漲跌價差"].values[0][0] != "0" else "X"

df= df.apply(lambda s:s.astype(str).str.replace(',',''))

# cols_to_numeric = ['開盤價', '最高價', '最低價', '收盤價', '漲跌價差']
# df[cols_to_numeric] = df[cols_to_numeric].apply(pd.to_numeric, errors='coerce')
# df[cols_to_numeric].fillna(0)


In [298]:
temp = df[df['證券代號'] == '8291']

cols_to_numeric = ['開盤價', '最高價', '最低價', '收盤價', '漲跌價差']
df[cols_to_numeric] = df[cols_to_numeric].apply(pd.to_numeric, errors='coerce')
df[cols_to_numeric] = df[cols_to_numeric].fillna(0)
df

,Date,證券代號,證券名稱,收盤價,漲跌價差,開盤價,最高價,最低價,成交股數,成交金額,成交筆數,漲跌(+/-)
99,20240514,1240,茂生農經,52.50,0.50,52.00,52.70,52.00,14405,756662,20,+
100,20240514,1259,安心,75.00,0.60,73.80,75.00,73.80,13102,978538,27,+
101,20240514,1264,德麥,306.00,0.00,306.00,306.00,305.50,9479,2900208,157,+
102,20240514,1268,漢來美食,167.00,-2.00,171.00,171.00,166.50,207820,34881444,279,+
103,20240514,1336,台翰,19.75,0.10,19.65,19.95,19.65,58469,1157155,48,+
...,...,...,...,...,...,...,...,...,...,...,...,...
11793,20240514,9949,琉園,23.10,0.10,23.05,23.10,23.00,226432,5208371,32,+
11794,20240514,9950,萬國通,15.80,-0.20,15.95,16.00,15.80,104390,1654310,89,+
11795,20240514,9951,皇田,70.60,-0.40,70.50,71.00,70.30,140922,9944787,188,+
11796,20240514,9960,邁達康,27.00,0.00,27.00,27.10,26.95,40239,1086557,36,+


In [300]:
df[df['證券代號'] == '8291']

,Date,證券代號,證券名稱,收盤價,漲跌價差,開盤價,最高價,最低價,成交股數,成交金額,成交筆數,漲跌(+/-)
11738,20240514,8291,尚茂,0.0,0.0,0.0,0.0,0.0,5184,28668,10,+
